# 워드클라우드 gradio 예제

In [5]:
import gradio as gr
from wordcloud import WordCloud
import matplotlib.pyplot as plt

#워드 클라우드 생성함수
def generate_wordcloud(file_obj):
    try:
        #파일이 없는 경우 처리
        if file_obj is None:
            return None;
        
        #Gradio의 파일 객체에서 파일 경로 가져오기
        file_path=file_obj.name;
        
        #파일읽기
        with open(file_path, "r", encoding="utf-8") as file:
            text=file.read();
            
        #워드클라우드 생성
        wordcloud=WordCloud(
            font_path="malgun",
            background_color="white",
            width=800,
            height=600,
            max_words=200,
            max_font_size=100,
            min_font_size=10,
            random_state=42
        ).generate(text);
        
        #matplotlib 그래프 초기화
        plt.clf();
        
        # 워드클라우드 이미지를 플롯
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.tight_layout()
        
        #결과 이미지를 저장
        output_path="wordcloud.png";
        plt.savefig(output_path);
        plt.close();    #메모리 누수 방지를 위해 figure 닫기
        
    except Exception as e:
        print(f"Error: {str(e)}");
        return None;

In [6]:
#Gradio 인터페이스 생성
iface=gr.Interface(
    fn=generate_wordcloud,
    inputs=gr.File(label="Upload a .txt file"),
    outputs=gr.Image(type="filepath", label="WordCloud")
)

In [7]:
iface.launch(server_port=7861, share=True, server_name="0.0.0.0");

* Running on local URL:  http://0.0.0.0:7861
* Running on public URL: https://f0e001ee1a9b38a487.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
iface.close()

Closing server running on port: 7861


# 자기소개서 도우미 챗봇 예제(Gradio)

In [32]:
#사전 설치: pip install fpdf
import os
import gradio as gr
from langchain.llms import Ollama
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from fpdf import FPDF

#Ollama 설정 (Gemma2 모델 사용)
os.environ["OLLAMA_API_BASE"]="http://localhost:11434"  # Ollama 서버 주소
ollama_model=Ollama(model="gemma2")

#다양한 템플릿 설정
TEMPLATES={
    "취업": "Based on the following keywords and example, write a personal statement for a job application",
    "대학원": "Using the provided keywords, draft a personal statement for a graduate school application",
    "봉사활동": "With the given keywords, write a personal statement emphasizeing volunteer experience and motivation"
}

#언어지원: 한국어, 영어, 일본어
LANGUAGES={
    "한국어": "Please write the response in Korean",
    "영어": "Please write the response in English",
    "일본어": "Please write the response in Japanese"
}


C:\Users\human-10\AppData\Local\Temp\ipykernel_7840\726246440.py:11: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  ollama_model=Ollama(model="gemma2")


In [36]:
#자동 키워드 추천함수
def recommend_keywords(purpose):
    if purpose=="취업":
        return "책임감, 팀워크, 문제 해결 능력"
    elif purpose=="대학원":
        return "연구 열정, 창의력, 학업 성취도"
    elif purpose=="봉사활동":
        return "사회적 책임감, 희생정신, 리더쉽"
    else:
        return ""


In [37]:
#자기소개서 작성함수
def generate_statement(purpose, language, keywords, example_sentence=None):
    if purpose not in TEMPLATES:
        return "❌ 지원 목적을 올바르게 선택해주세요."
    if purpose not in LANGUAGES:
        return "❌ 언어를 올바르게 선택해주세요."
    
    #템플릿 생성
    template=TEMPLATES[purpose]+"\n\nKeywords: {keywords}\n"+LANGUAGES[language];
    if example_sentence:
        template+=f"\n\nExample sentence:{example_sentence}";
        
    prompt=PromptTemplate(input_variables=["keywords"], template=template);
    chain=LLMChain(llm=ollama_model, prompt=prompt)
    response=chain.run({"keywords":keywords})
    return response

In [38]:
#PDF 저장함수
def save_to_pdf(statement, filename="personal_statement.pdf"):
    pdf=FPDF();
    pdf.add_page();
    pdf.add_font("MalgunGothic","r","C:\Windows\Fonts\malgun.ttf",uni=True);
    pdf.set_font("MalgunGothic", size=12);
    pdf.multi_cell(0, 10, statement);
    pdf.output(filename);
    return f"✔️ PDF 저장 완료: {filename}"

In [41]:
#Gradio 인터페이스
def chatbot_interface(purpose, language, keywords, example_sentence=None, save_pdf=False):
    statement=generate_statement(purpose, language, keywords, example_sentence)
    if save_pdf:
        save_to_pdf(statement)
    return statement

In [42]:
with gr.Blocks() as demo:
    gr.Markdown("# 📝 다목적 자기소개서 작성 도우미")
    gr.Markdown("키워드와 추천 문장을 활용하여 취업, 대학원, 봉사활동 자기소개서를 생성하고 PDF로 저장하세요!")

    # 입력 영역
    with gr.Row():
        purpose_input = gr.Dropdown(label="지원 목적", choices=["취업", "대학원", "봉사활동"], value="취업")
        language_input = gr.Dropdown(label="언어 선택", choices=["한국어", "영어", "일본어"], value="한국어")

    recommended_keywords = gr.Textbox(label="추천 키워드", interactive=False)
    recommend_btn = gr.Button("키워드 추천")
    recommend_btn.click(recommend_keywords, inputs=[purpose_input], outputs=[recommended_keywords])

    with gr.Row():
        keywords_input = gr.Textbox(label="사용자 키워드 입력", placeholder="예: 책임감, 팀워크, 문제 해결 능력")
        example_sentence_input = gr.Textbox(
            label="추천 문장 (선택 사항)",
            placeholder="예: '저는 도전을 두려워하지 않고 성공적으로 프로젝트를 완수했습니다.'"
        )

    save_pdf_toggle = gr.Checkbox(label="PDF로 저장", value=False)

    # 출력 영역
    output = gr.Textbox(label="작성된 자기소개서", lines=6)
    submit_btn = gr.Button("작성하기")
    submit_btn.click(
        fn=chatbot_interface,
        inputs=[purpose_input, language_input, keywords_input, example_sentence_input, save_pdf_toggle],
        outputs=[output]
    )

In [43]:
#실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "d:\241223\aiProject2\.venv\lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "d:\241223\aiProject2\.venv\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "d:\241223\aiProject2\.venv\lib\site-packages\gradio\blocks.py", line 2047, in process_api
    result = await self.call_function(
  File "d:\241223\aiProject2\.venv\lib\site-packages\gradio\blocks.py", line 1594, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "d:\241223\aiProject2\.venv\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "d:\241223\aiProject2\.venv\lib\site-packages\anyio\_backends\_asyncio.py", line 2461, in run_sync_in_worker_thread
    return await future
  File "d:\241223\aiProject2\.venv\lib\site-pack

In [35]:
#종료
demo.close()

Closing server running on port: 7860
